In [1]:
# import key from non-tracked file
from openai import OpenAI
import os
import matplotlib.image as img
import requests
import base64
from PIL import Image
import io

## Definimos la API KEY
la key esta definida en key.py pero pues esta untracked porque si se sube a GitHub hay bots que roban credenciales

In [2]:
from key import key
key = key()
os.environ["OPENAI_API_KEY"] = key

## Metodo para codificar la imagen y forzarla a 256x256

In [3]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        image = Image.open(image_file)
        
        # Resize the image to 256x256
        image = image.resize((256, 256))
        
        # Save the resized image to a bytes buffer
        buffer = io.BytesIO()
        image.save(buffer, format="JPEG")  # You can change the format if needed
        
        byte_data = buffer.getvalue()
        string = base64.b64encode(byte_data).decode('utf-8')
        
        return string

# Input

In [4]:
# Imagen del Agave
image_path = "agave.jpg"

# Prompt
text_prompt = "Utiliza la imagen y descripción adjuntas para informar a los patrocinadores sobre el estado mensual de su planta de agave en peligro de extinción. Describe el crecimiento y cambios notables de la planta. Finaliza con un agradecimiento sincero por su continuo apoyo en este boletín mensual.Genera un mensaje amable para actualizar a los patrocinadores sobre el estado de su planta de agave en peligro de extinción. Incluye agradecimiento por su apoyo."

# Codificar imagen
base64_image = encode_image(image_path)

# JSON payload
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {key}"
}

payload = {
  "model": "gpt-4-vision-preview",
  "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": text_prompt
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
  ],
  "max_tokens": 400
}


## Hacemos el request
Cuesta aprox 0.01 USD

In [5]:
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

# Output
Costo del request

In [6]:
# Cost 0.01 per 1K tokens input and 0.03 per 1K tokens output
input_cost = response.json()['usage']['prompt_tokens']/1000 * 0.01
output_cost = response.json()['usage']['completion_tokens']/1000 * 0.03

total_cost = input_cost + output_cost

total_cost


0.01499

## Mensaje generado

In [7]:
text = response.json()['choices'][0]['message']['content']

text = text.split('\n')

for i in text:
    print(i)


Queridos patrocinadores,

Estamos encantados de compartir con ustedes la última actualización sobre la condición de su querida planta de agave en peligro de extinción. Como pueden ver en la imagen adjunta, la planta muestra un crecimiento vigoroso y saludable. Sus hojas han mantenido una rica coloración verde, con las líneas blancas características que resaltan su bella geometría natural. No hemos observado signos de enfermedades ni de plagas, lo cual es un testimonio de las condiciones ideales bajo las que se mantiene y el cuidado meticuloso que recibe.

Es más, nos complace notificarles que la planta ha aumentado ligeramente en tamaño desde nuestra última actualización, una señal positiva de que se está adaptando y prosperando en su entorno. Hemos tomado medidas adicionales para asegurar que la planta tenga el equilibrio perfecto de luz solar y riego, lo cual es crucial para su desarrollo.

Su involucramiento y apoyo continuo son fundamentales para la conservación de estas magníficas